In [6]:
!pip install python-docx
!pip install PyPDF2
!pip install pdf2image
!pip install pytesseract
!pip install opencv-python
!pip install date-extractor
!pip install PyMuPDF

In [81]:

df_resumes = df_texts.copy()
import re

def preprocess_text(text):
    """
    Preprocess the resume text to prepare for entity recognition.
    The function performs the following tasks:
    - Removes unnecessary whitespace
    - Standardizes newline characters
    """
    # Replace various types of whitespace with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    return text

# Apply preprocessing to the resume texts
df_resumes['Cleaned_Text'] = df_resumes['Text'].apply(preprocess_text)

# Show the first few entries of the cleaned text to ensure it's been processed correctly
df_resumes[['Cleaned_Text']]


,Cleaned_Text
0,"Hovhannes Mkrtchyan GGeyerstrasse 52, 80469, M..."
1,Judith Kuneth Frontend Developer | UX Designer...
2,Nicholas Jedamzik njedamzik@jedamzicjedamaik.d...
3,DARKO KESEGI MSc of Economics shrkokesegi@ygma...
4,QUALIFIKATIONSPROFIL BENEDIKT JERAT ®. SENIOR ...
5,SAMI RAUSCH ‘CR Consulting Rausch Management C...
6,Awe: Giese 60 ‘eat sin Mirko Neumann Telefon: ...
7,LEBENSLAUF Eunsu AHN PERSONLICHE DATEN Adresse...
8,Top Itservices AG Borhane Cheikh Ahmed Brigelm...
9,Judith Kuneth Frontend Developer | UX Designer...


In [82]:
import spacy

# Load a pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract entities using spaCy
def extract_entities(text):
    doc = nlp(text)
    entities = {'names': [], 'phones': [], 'emails': []}
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            entities['names'].append(ent.text)

    emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    phones = re.findall(r'\+?[\d\s]{8,13}', text)  
    if emails:
        entities['emails'].extend(emails)
    if phones:
        entities['phones'].extend(phones)
    return entities

sample_resumes = df_resumes['Cleaned_Text'].apply(extract_entities)

df_resumes


,Document,Text,Cleaned_Text
0,1034151_page,"Hovhannes Mkrtchyan\n\nGGeyerstrasse 52, 80469...","Hovhannes Mkrtchyan GGeyerstrasse 52, 80469, M..."
1,1037402_page,Judith Kuneth\n\nFrontend Developer | UX Desig...,Judith Kuneth Frontend Developer | UX Designer...
2,1039995_page,Nicholas Jedamzik\n\nnjedamzik@jedamzicjedamai...,Nicholas Jedamzik njedamzik@jedamzicjedamaik.d...
3,1042050_page,DARKO\nKESEGI\n\nMSc of Economics\n\nshrkokese...,DARKO KESEGI MSc of Economics shrkokesegi@ygma...
4,1042140_page,QUALIFIKATIONSPROFIL\nBENEDIKT JERAT\n\n®. SEN...,QUALIFIKATIONSPROFIL BENEDIKT JERAT ®. SENIOR ...
5,1042449_page,SAMI RAUSCH\n\n‘CR Consulting Rausch\n\nManage...,SAMI RAUSCH ‘CR Consulting Rausch Management C...
6,1042894_page,Awe: Giese 60\n‘eat sin\nMirko Neumann Telefon...,Awe: Giese 60 ‘eat sin Mirko Neumann Telefon: ...
7,1043487_page,LEBENSLAUF Eunsu AHN\n\nPERSONLICHE DATEN\nAdr...,LEBENSLAUF Eunsu AHN PERSONLICHE DATEN Adresse...
8,1044442_page,Top Itservices AG Borhane Cheikh Ahmed\n\nBrig...,Top Itservices AG Borhane Cheikh Ahmed Brigelm...
9,example2_page,Judith Kuneth\n\nFrontend Developer | UX Desig...,Judith Kuneth Frontend Developer | UX Designer...


In [84]:
import pandas as pd
import re

# Load the dataframe from the uploaded CSV file
df_resumes = df_texts.copy()


def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace various types of whitespace with a single space
    text = text.strip()  # Remove leading/trailing whitespace
    return text


df_resumes['Cleaned_Text'] = df_resumes['Text'].apply(preprocess_text)


email_pattern = r'[\w.%+-]+[\s]*@[\s]*[\w.-]+[\s]*\.\s*[\w.%+-]+'

phone_pattern_improved = r'\+?(?:\(\d+\)\s*|\d+[\s-]*)?\d+[\s-]*\d+[\s-]*\d+(?:[\s-]*\d+)*'


def extract_name_heuristic(text):
    # Assuming the name is at the beginning and capitalized
    possible_names = re.findall(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*', text)
    return possible_names[:2]  # Return the first two matches as possible names


def extract_email_phone_improved(text):
    emails = re.findall(email_pattern, text)
    phones = re.findall(phone_pattern_improved, text)
    phones = [re.sub(r'[^\d+]', '', phone) for phone in phones]
    return emails, phones


df_resumes['Extracted_Names'] = df_resumes['Cleaned_Text'].apply(extract_name_heuristic)
df_resumes['Extracted_Emails'], df_resumes['Extracted_Phones'] = zip(*df_resumes['Cleaned_Text'].map(extract_email_phone_improved))


df_resumes


,Document,Text,Cleaned_Text,Extracted_Names,Extracted_Emails,Extracted_Phones
0,1034151_page,"Hovhannes Mkrtchyan\n\nGGeyerstrasse 52, 80469...","Hovhannes Mkrtchyan GGeyerstrasse 52, 80469, M...","[Hovhannes Mkrtchyan, Munich]",[],"[80469, 44917677512727, 2021, 2019, 2021, 2018..."
1,1037402_page,Judith Kuneth\n\nFrontend Developer | UX Desig...,Judith Kuneth Frontend Developer | UX Designer...,"[Judith Kuneth Frontend Developer, Designer Bi...","[judith.kuneth@gmail.com, judith.kuneth@gmail....","[1993, +4367683555785, 202, 2019, 2020, 369, 8..."
2,1039995_page,Nicholas Jedamzik\n\nnjedamzik@jedamzicjedamai...,Nicholas Jedamzik njedamzik@jedamzicjedamaik.d...,"[Nicholas Jedamzik, Lbeck]","[njedamzik@jedamzicjedamaik.de, njedamzik@jeda...","[0451, 58609041, 2022, 2022, 2021, 2022, 2020,..."
3,1042050_page,DARKO\nKESEGI\n\nMSc of Economics\n\nshrkokese...,DARKO KESEGI MSc of Economics shrkokesegi@ygma...,"[Economics, Sod]",[],"[2019, 2020, 201572019, 20112015, 2022, 2021, ..."
4,1042140_page,QUALIFIKATIONSPROFIL\nBENEDIKT JERAT\n\n®. SEN...,QUALIFIKATIONSPROFIL BENEDIKT JERAT ®. SENIOR ...,"[Einsatzschwerpunkte, Schwerpunkte]",[],"[2022, 2022, 2022, 2022, 2022, 2022, 2020, 202..."
5,1042449_page,SAMI RAUSCH\n\n‘CR Consulting Rausch\n\nManage...,SAMI RAUSCH ‘CR Consulting Rausch Management C...,[Consulting Rausch Management Co Program Manag...,[],"[0000, 828, 112022, 0000, 828, 1120222022, 120..."
6,1042894_page,Awe: Giese 60\n‘eat sin\nMirko Neumann Telefon...,Awe: Giese 60 ‘eat sin Mirko Neumann Telefon: ...,"[Awe, Giese]",[NeumannMisko@smx.te],"[03080867960, 1975, 12621, +490, 3050567980, +..."
7,1043487_page,LEBENSLAUF Eunsu AHN\n\nPERSONLICHE DATEN\nAdr...,LEBENSLAUF Eunsu AHN PERSONLICHE DATEN Adresse...,"[Eunsu, Adresse Olgastaie]",[],"[70180, 018758219873, 1091, 2022, 091201510, 2..."
8,1044442_page,Top Itservices AG Borhane Cheikh Ahmed\n\nBrig...,Top Itservices AG Borhane Cheikh Ahmed Brigelm...,"[Top Itservices, Borhane Cheikh Ahmed Brigelma...",[],"[34850679, 64285, 017672310353, 04122022, 3485..."
9,example2_page,Judith Kuneth\n\nFrontend Developer | UX Desig...,Judith Kuneth Frontend Developer | UX Designer...,"[Judith Kuneth Frontend Developer, Designer Bi...","[judith.kuneth@gmail.com, judith.kuneth@gmail....","[1993, +4367683555785, 202, 2019, 2020, 369, 8..."


# Tesseract

In [20]:
# Set the Tesseract command path after installation
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

## For single pdf

In [21]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
pdf_path = r'C:\Users\Aveno\Documents\Resume\cv\1042050.pdf'
images = convert_from_path(pdf_path)
for i, image in enumerate(images):
    text = pytesseract.image_to_string(image)


In [22]:
text

'DARKO\n\nKESEGI\n\nMSc of Economics\n\nBIRTH DATE:\n15.02.1996.\n\nCITY:\nSubotica, Serbia\n\nEMAIL:\ndarkokesegi@gmail.com\n\nNUMBER:\n+381 60 0552 608\n\n+381 63 596 383 G=>\n\nLinkedin:\nwww.linkedin.com/in/darko-kesegi\n\ne EDUCATION\n\nFaculty of Economics in Subotica, University\nof Novi\n\nSad\n\n“Master studies\n\nDirection: Finance, Banking and Insurance\n-enrolled / finished: 2019 /2020\n“Undergraduate\n\n-enrolled / finished: 2015/2019\n\nHigh School of Economics "Bosa Miliéevi¢",\nSubotica\n\n-direction: economic technician\n-enrolled / finished: 2011/2015\n\n« LANGUAGES\n- Serbian - Native\n- English - Advanced\n\n- German - Intermediate\n\n¢ CERTIFICATION\n-ECDL certificate\n-Excel: Advanced formulas and Functions\n(Linkedin learning)\n\n¢ PERSONAL PROFILE\n| think for myself that | am very organized, responsible, persistent and hardworking person.\n| am able to handle multiple tasks in daily basis. | shown myself to be a person capable of\ncooperating with people of var

## For all pdfs in the path

In [23]:
import os
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import pandas as pd


pdf_dir = r'C:\Users\Aveno\Documents\Resume\cv\pdf'
pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
df = pd.DataFrame(columns=['filename', 'text'])


for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    images = convert_from_path(pdf_path)
    text_content = ''
    for image in images:
        text = pytesseract.image_to_string(image)
        text_content += text + ' '  # Add a space as a separator between pages

    new_row = pd.DataFrame({'filename': [pdf_file], 'text': [text_content]})
    df = pd.concat([df, new_row], ignore_index=True)

df


,filename,text
0,1034151.pdf,"Hovhannes Mkrtchyan\n\nGeyerstrasse 52, 80469,..."
1,1037402.pdf,"Birthday: August 26th, 1993\n\nHome base: Inns..."
2,1039995.pdf,Nicholas Jedamzik\n\nn.jedamzik@jedamzik-jedam...
3,1042050.pdf,DARKO\n\nKESEGI\n\nMSc of Economics\n\nBIRTH D...
4,1042140.pdf,Einsatzschwerpunkte\n\nConsulting\n\nMentoring...
5,1042449.pdf,SAMI RAUSCH\n\nManagement Consultant\n\nPERSON...
6,1042894.pdf,Adresse:\n\nGiesestraBe 60\n12621 Berlin\n\nMi...
7,1043487.pdf,LEBENSLAUF Eunsu AHN\n\nPERSONLICHE DATEN\n\nA...
8,1044442.pdf,Top Itservices AG Borhane Cheikh Ahmed\n\nBrii...
9,example.pdf,Lebenslauf Lisa Maria Zettier\n\nPersonliche D...


In [3]:
# df.to_csv('df_tesseract.csv')
df = pd.read_csv('df_tesseract.csv')

In [4]:
import spacy
import pandas as pd
import re


nlp = spacy.load('en_core_web_sm')
phone_regex = re.compile(r'\b(?:\+?\d{1,3})?\s*(?:\(\d{2,3}\)\s*)?\d{3}[-\s]?\d{2,3}[-\s]?\d{2,4}\b')
email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')


def extract_phone_email(text):
    phones = phone_regex.findall(text)
    emails = email_regex.findall(text)
    return phones, emails

def extract_emails_with_spacy(text):
    doc = nlp(text)
    return [token.text for token in doc if token.like_email]

def extract_names(text):
    doc = nlp(text)
    names = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    return names

df['phones'], df['emails'] = zip(*df['text'].apply(extract_phone_email))
df['emails_spacy'] = df['text'].apply(extract_emails_with_spacy)
df['names'] = df['text'].apply(extract_names)

df


,Unnamed: 0,filename,text,phones,emails,emails_spacy,names
0,0,1034151.pdf,"Hovhannes Mkrtchyan\n\nGeyerstrasse 52, 80469,...",[176 77512727],[mkrtchyan.hov.hm@gmail.com],[mkrtchyan.hov.hm@gmail.com],"[Hovhannes Mkrtchyan, UniBank OJSC\nLoan, Tax ..."
1,1,1037402.pdf,"Birthday: August 26th, 1993\n\nHome base: Inns...",[43 676 83555],"[juergen.hahn@gisolutions.at, markus.salmhofer...","[juergen.hahn@gisolutions.at, markus.salmhofer...","[g.\nZz, Judith Kuneth\n\n, Testing Enginner, ..."
2,2,1039995.pdf,Nicholas Jedamzik\n\nn.jedamzik@jedamzik-jedam...,[58609041],[n.jedamzik@jedamzik-jedamzik.de],[n.jedamzik@jedamzik-jedamzik.de],"[Nicholas Jedamzik, Deutschland, LinkedIn ¢ Xi..."
3,3,1042050.pdf,DARKO\n\nKESEGI\n\nMSc of Economics\n\nBIRTH D...,"[381 60 0552, 381 63 596]",[darkokesegi@gmail.com],[darkokesegi@gmail.com],"[DARKO\n\nKESEGI, Subotica, Functions, Linkedi..."
4,4,1042140.pdf,Einsatzschwerpunkte\n\nConsulting\n\nMentoring...,[2021-2022],"[info@digitalfrontiers.de, info@digitalfrontie...","[info@digitalfrontiers.de, info@digitalfrontie...","[Programmieren, Agile Vorgehensweisen, Cloud K..."
5,5,1042449.pdf,SAMI RAUSCH\n\nManagement Consultant\n\nPERSON...,[49 170 174 58],[sami.rausch@consulting-rausch.com],[sami.rausch@consulting-rausch.com],"[Ludwigshafen, LANGUAGUES, Ludwigshafen am Rhe..."
6,6,1042894.pdf,Adresse:\n\nGiesestraBe 60\n12621 Berlin\n\nMi...,"[030-50 56, 030-50 56, 030-50 56, 030-50 56, 0...","[NeumannMirko@gmx.de, NeumannMirko@gmx.de, Neu...","[NeumannMirko@gmx.de, NeumannMirko@gmx.de, Neu...","[Mirko Neumann Telefon, Mirko Neumann, Juli, M..."
7,7,1043487.pdf,LEBENSLAUF Eunsu AHN\n\nPERSONLICHE DATEN\n\nA...,[157 58219873],[eunsuahn.pro@gmail.com],[eunsuahn.pro@gmail.com],"[Telefon, Staatsangehorigkeit Koreanisch, Stid..."
8,8,1044442.pdf,Top Itservices AG Borhane Cheikh Ahmed\n\nBrii...,"[176 72310353, 100\n64295, 100\n64295, 100\...","[stbochei@stud.h-da.de, freitag@eit.h-da.de, i...","[stbochei@stud.h-da.de, freitag@eit.h-da.de, i...","[Kdln, Darmstadt, Stelle, Softwareentwickler, ..."
9,9,example.pdf,Lebenslauf Lisa Maria Zettier\n\nPersonliche D...,[],[],[],"[Lebenslauf, Lisa Maria Zettier, Personliche D..."


In [26]:
df['text'][0]

'Hovhannes Mkrtchyan\n\nGeyerstrasse 52, 80469, Munich, Germany\n+49 176 77512727\nmkrtchyan.hov.hm@gmail.com\nEXPERIENCE\n\nEPAM Systems GmbH\nSep 2021 — Present Software Testing Engineer\n* Test software to verify it meets the predefined requirements\n* Ensure the quality of the system and the DB as a whole\n* Detect bug issues, develop test cases and implement regression tests\n* Prepare file defect reports and report test progress\n* Support in automated tests and their optimization\n\nACRA Credit Reporting Credit Bureau\nJan 2019 — Aug 2021 Data Quality Assurance Engineer\n* Identify and analyze the bugs, work with software developers to fix issues\n* Elaborate complex test plans and execute manual testing\n* Design product improvement suggestions\n\nJan 2018 — Jan 2019 Operational Systems Development Expert\n* Develop technical and functional requirements for ACRA software system\n* Plan, develop and test software systems for software enhancements\n* Handle database systems and i